# Azure AI Agent with Bing Grounding Example

This notebook demonstrates how to create an Azure AI agent that uses Bing Grounding search to find real-time information from the web.

## Features Covered:
- Setting up Bing Grounding search integration
- Using HostedWebSearchTool for web searches
- Creating agents that can access current web information
- Proper resource management with async context managers

## Prerequisites

Before running this notebook, ensure you have:
- Azure CLI installed and authenticated (`az login --use-device-code`)
- Access to an Azure AI Foundry project with deployed models
- Environment variables set up (see Initial Setup section below)

This example demonstrates Azure AI agents with Bing Grounding for web search capabilities. The agent can search the web for current information and provide informed responses.

## Import Libraries

Import the required libraries for Azure AI agent functionality.

In [ ]:
import os
from pathlib import Path
from agent_framework import ChatAgent, HostedWebSearchTool
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential

## Initial Setup

Initialize the Azure AI project client and set up authentication.

In [ ]:
from pathlib import Path  # For working with file paths
import os  # For environment variables
import time  # For sleep function
from dotenv import load_dotenv  # For loading environment variables from .env file
from azure.identity import InteractiveBrowserCredential  # For Azure authentication
from azure.ai.projects import AIProjectClient  # Import the AIProjectClient

# Get the path to the .env file which is in the base folder (2 levels up)
notebook_path = Path().absolute()  # Get absolute path of current notebook
parent_dir = notebook_path.parent  # Get parent directory
load_dotenv('../../.env')  # Load environment variables from .env file in base folder

# Get tenant ID for authentication
tenant_id = os.environ.get("TENANT_ID")
conn_string = os.environ.get("PROJECT_CONNECTION_STRING")
if not conn_string:
    conn_string = "https://foundrye2elab.services.ai.azure.com/api/projects/banff"

print(f"🔑 Using Tenant ID: {tenant_id}")

# Initialize the AI Project Client using simplified browser-based authentication
try:
    print("🌐 Using browser-based authentication to bypass Azure CLI cache issues...")
    
    # Use only InteractiveBrowserCredential with the specific tenant
    credential = InteractiveBrowserCredential(tenant_id=tenant_id)
    
    # Create the project client using the endpoint
    project_client = AIProjectClient(
        endpoint=conn_string,
        credential=credential
    )
    print("✅ Successfully initialized AIProjectClient")
except Exception as e:
    # Print error message if client initialization fails
    print(f"❌ Error initializing project client: {str(e)}")
    print("💡 Please complete the browser authentication prompt that should appear")

## Check Environment Setup

Let's verify that the required environment variable is set:

In [ ]:
# Load environment variables
load_dotenv('../../../.env')

# Get the Bing connection ID from environment
bing_connection_id = os.getenv('BING_CONNECTION_ID')

if bing_connection_id:
    # Fix the connection ID format to include the project path
    # The current format is missing the /projects/<project_name> part
    # Extract project name from PROJECT_CONNECTION_STRING
    project_conn_string = os.getenv('PROJECT_CONNECTION_STRING')
    if project_conn_string:
        # Extract project name from the URL (e.g., "banff" from "https://...ai.azure.com/api/projects/banff")
        project_name = project_conn_string.split('/projects/')[-1]
        
        # Insert the project path into the connection ID
        # Current: /subscriptions/.../accounts/foundrye2elab/connections/bingprval
        # Target:  /subscriptions/.../accounts/foundrye2elab/projects/banff/connections/bingprval
        bing_connection_id = bing_connection_id.replace('/connections/', f'/projects/{project_name}/connections/')
    
    print(f"✅ BING_CONNECTION_ID loaded and corrected successfully")
    print(f"Connection ID: {bing_connection_id[:50]}...")  # Show first 50 chars for security
else:
    print("❌ BING_CONNECTION_ID environment variable is not set")
    print("Please set this variable before proceeding.")

## Create Bing Grounding Search Tool

The `HostedWebSearchTool` allows our agent to perform web searches using Bing. The connection ID will be automatically picked up from the environment variable:

In [ ]:
# Create Bing Grounding search tool using HostedWebSearchTool
# Pass the connection_id explicitly from the environment variable
bing_search_tool = HostedWebSearchTool(
    name="Bing Grounding Search",
    description="Search the web for current information using Bing",
    connection_id=bing_connection_id,
)

print("✅ Bing search tool created successfully")

## Create and Run the Agent

Now let's create an Azure AI agent that can use the Bing search tool to find current information from the web:

In [ ]:
async def main() -> None:
    """Main function demonstrating Azure AI agent with Bing Grounding search."""
    
    # Use AzureAIAgentClient with proper configuration
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(
            project_endpoint=conn_string,  # Use project_endpoint parameter
            async_credential=credential,
            model_deployment_name="gpt-4o"  # Specify the model deployment name
        ).create_agent(
            name="BingSearchAgent",
            instructions=(
                "You are a helpful assistant that can search the web for current information. "
                "Use the Bing search tool to find up-to-date information and provide accurate, "
                "well-sourced answers. Always cite your sources when possible."
            ),
            tools=bing_search_tool,
        ) as agent,
    ):
        # Demonstrate agent capabilities with web search
        print("=== Azure AI Agent with Bing Grounding Search ===\n")

        user_input = "What is the most popular programming language?"
        print(f"User: {user_input}")
        response = await agent.run(user_input)
        print(f"Agent: {response}\n")

## Execute the Example

Run the main function to see the agent in action:

In [ ]:
# Run the main function
await main()

## Try Additional Queries

Let's test the agent with more queries that require current information:

In [ ]:
async def test_more_queries():
    """Test the agent with additional queries that require web search."""
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(
            project_endpoint=conn_string,  # Use project_endpoint parameter
            async_credential=credential,
            model_deployment_name="gpt-4o"  # Specify the model deployment name
        ).create_agent(
            name="BingSearchAgent",
            instructions=(
                "You are a helpful assistant that can search the web for current information. "
                "Use the Bing search tool to find up-to-date information and provide accurate, "
                "well-sourced answers. Always cite your sources when possible."
            ),
            tools=bing_search_tool,
        ) as agent,
    ):
        queries = [
            "What are the latest trends in artificial intelligence?",
            "What is the current weather in Seattle?",
            "What are the latest Microsoft announcements?"
        ]
        
        for query in queries:
            print(f"\n--- Query: {query} ---")
            response = await agent.run(query)
            print(f"Agent: {response}")
            print("-" * 50)

# Run additional queries
await test_more_queries()

## Key Takeaways

1. **Bing Grounding Integration**: The `HostedWebSearchTool` provides seamless integration with Bing search capabilities
2. **Environment Configuration**: The `BING_CONNECTION_ID` environment variable is crucial for connecting to your Azure AI project's Bing resource
3. **Real-time Information**: Agents can now access current web information, making them more useful for time-sensitive queries
4. **Source Citation**: The agent is instructed to cite sources, promoting transparency and reliability
5. **Resource Management**: Using async context managers ensures proper cleanup of resources
6. **Tool Integration**: The Bing search tool seamlessly integrates with the agent's existing capabilities

## Troubleshooting

If you encounter issues:
1. **Authentication**: Ensure you're logged in with `az login`
2. **Environment Variable**: Double-check that `BING_CONNECTION_ID` is correctly set
3. **Azure AI Project**: Verify that Bing Grounding is properly configured in your Azure AI Foundry project
4. **Permissions**: Ensure you have the necessary permissions to access the connected Bing search resource